In [1]:
import pandas as pd
import logging as lg

In [2]:
logger = lg.getLogger(__name__) #new logger
logger.setLevel(lg.ERROR)
logger.setLevel(lg.INFO)

formatter = lg.Formatter(' %(name)s : %(asctime)s : %(levelname)s : %(message)s')

filehandler= lg.FileHandler('Pandas_CW.log')
filehandler.setFormatter(formatter)

logger.addHandler(filehandler)

stream_handler = lg.StreamHandler() #no need to set log level as its set to error by logger

logger.addHandler(stream_handler)
#lg.basicConfig(filename = '{}.log'.format(__name__), level = lg.INFO,format = '%(asctime)s : %(levelname)s : %(message)s')

# Dataset: ONE - chipotle.tsv

In [3]:
chipotle_df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv",sep='\t')

In [4]:
chipotle_df.columns

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

In [5]:
dic = {i:chipotle_df[i].isnull().sum() * 100 / len(chipotle_df) for i in chipotle_df.columns}
chipotle_df_null = pd.DataFrame(dic.items(),columns=['chipotle_df_col','sum_null']).sort_values('sum_null',ascending=False)
"""
chipotle_df_null = pd.DataFrame.from_dict(dic.items())
chipotle_df_null.columns = ['chipotle_df_col','sum_null']
chipotle_df_null
"""
chipotle_df_null

,chipotle_df_col,sum_null
3,choice_description,26.958027
0,order_id,0.000000
1,quantity,0.000000
2,item_name,0.000000
4,item_price,0.000000


In [6]:
chipotle_null_data = chipotle_df[chipotle_df['choice_description'].isnull()==True]
chipotle_null_data.reset_index(drop=True).head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
2,3,1,Side of Chips,NaN,$1.69
3,5,1,Chips and Guacamole,NaN,$4.45
4,7,1,Chips and Guacamole,NaN,$4.45


In [7]:
chipotle_df['choice_description'].isnull().sum() * 100 / len(chipotle_df)

26.958026828212894

## Functions for Null Count and Percentage

In [8]:
def missing_cnt_pt_sorted(df):
    try:
        import pandas as pd
        dic_sum = {i:sum(df[i].isnull()) for i in df.columns}
        dic_prcnt = {i:sum(df[i].isnull()* 100 / len(df)) for i in df.columns}
        df1_null_ct = pd.DataFrame(dic_sum.items(),columns=['df_col','sum_null']).sort_values('sum_null',ascending=False)
        df1_null_pt = pd.DataFrame(dic_prcnt.items(),columns=['df_col','percent_null']).sort_values('percent_null',ascending=False)
        df_missing_cnt_pt = df1_null_ct.join(df1_null_pt['percent_null'])
        #df_missing_cnt_pt = pd.concat([df1_null_ct,df1_null_pt['percent_null']],axis=1)
        df_missing_cnt_pt.reset_index(drop=True,inplace=True)
        return df_missing_cnt_pt
    except Exception as e:
        logger.error("error : ",e)

In [9]:
chipotle_missing = missing_cnt_pt_sorted(chipotle_df)
chipotle_missing

,df_col,sum_null,percent_null
0,choice_description,1246,26.958027
1,order_id,0,0.000000
2,quantity,0,0.000000
3,item_name,0,0.000000
4,item_price,0,0.000000


## Seperate null and detailed data as csv

In [10]:
null_index = chipotle_df[chipotle_df['choice_description'].isnull() == True].index
null_index

Int64Index([   0,    3,    6,   10,   14,   15,   20,   25,   30,   32,
            ...
            4582, 4584, 4588, 4596, 4598, 4600, 4605, 4613, 4614, 4616],
           dtype='int64', length=1246)

In [11]:
chipotle_null_data = chipotle_df[chipotle_df['choice_description'].isnull() == True].reset_index(drop=True)
chipotle_null_data.head()
#chipotle_null_data.to_csv("chipotle_null_data.csv")

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
2,3,1,Side of Chips,NaN,$1.69
3,5,1,Chips and Guacamole,NaN,$4.45
4,7,1,Chips and Guacamole,NaN,$4.45


In [12]:
chipotle_full_data = chipotle_df[chipotle_df['choice_description'].isnull() == False].reset_index(drop=True)
chipotle_full_data.head()
#chipotle_full_data.to_csv("chipotle_full_data.csv")

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Izze,[Clementine],$3.39
1,1,1,Nantucket Nectar,[Apple],$3.39
2,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
3,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
4,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75


In [13]:
def df_null_detailed_csv(df,colname):
    """
    Two Inputs req: df_name, column
    """
    try:
        import pandas as pd
        df_null_data = df[df[colname].isnull() == True].reset_index(drop=True)
        df_full_data = df[df[colname].isnull() == False].reset_index(drop=True)
        return df_null_data,df_full_data
    except Exception as e:
        logger.error("error : ",e)

In [14]:
chipotle_null_data,chipotle_full_data = df_null_detailed_csv(chipotle_df,"choice_description")

In [15]:
type(chipotle_null_data)

pandas.core.frame.DataFrame

## Data Processing

In [16]:
def preprocess_chipotle(df):
    try:
        df['item_price'] = df['item_price'].str.replace('$','',regex=True).astype(float) #data manipulation
        df['Total_Price'] = df['quantity']*df['item_price'] #new Column
        df['choice_description'] = df['choice_description'].str.replace('[','',regex=True).replace(']','',regex=True)
        process_choice_description = df['choice_description'].str.split(',',expand=True)
        process_choice_description.rename(columns={i:f'choice_{i+1}' for i in process_choice_description.columns},inplace=True)
        df_full_data_processed = pd.concat([df,process_choice_description],axis=1)
        df_full_data_processed.drop('choice_description',axis=1,inplace=True)
        return df_full_data_processed
    except Exception as e:
        logger.error("error : ",e)

In [17]:
chipotle_cleaned = preprocess_chipotle(chipotle_full_data)

In [18]:
chipotle_cleaned.head()

,order_id,quantity,item_name,item_price,Total_Price,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,choice_10
0,1,1,Izze,3.39,3.39,Clementine,None,None,None,None,None,None,None,None,None
1,1,1,Nantucket Nectar,3.39,3.39,Apple,None,None,None,None,None,None,None,None,None
2,2,2,Chicken Bowl,16.98,33.96,Tomatillo-Red Chili Salsa (Hot),Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None
3,3,1,Chicken Bowl,10.98,10.98,Fresh Tomato Salsa (Mild),Rice,Cheese,Sour Cream,Guacamole,Lettuce,None,None,None,None
4,4,1,Steak Burrito,11.75,11.75,Tomatillo Red Chili Salsa,Fajita Vegetables,Black Beans,Pinto Beans,Cheese,Sour Cream,Guacamole,Lettuce,None,None


In [19]:
chipotle_cleaned['item_name'].nunique()

38

In [20]:
def val_ct_pt(df,colname):
    c = df[colname].value_counts(dropna=False)
    p = df[colname].value_counts(dropna=False, normalize=True)
    c_p = pd.concat([c,p], axis=1, keys=['counts', '%'])
    return c_p

In [21]:
val_ct_pt(chipotle_cleaned,colname='item_name')

,counts,%
Chicken Bowl,726,0.215047
Chicken Burrito,553,0.163803
Steak Burrito,368,0.109005
Canned Soft Drink,301,0.089159
Steak Bowl,211,0.062500
Chicken Soft Tacos,115,0.034064
Chicken Salad Bowl,110,0.032583
Canned Soda,104,0.030806
Veggie Burrito,95,0.028140
Barbacoa Burrito,91,0.026955


In [22]:
chipotle_cleaned['item_name'].unique() #array o/p
#set(chipotle_cleaned['item_name']) #set o/p

array(['Izze', 'Nantucket Nectar', 'Chicken Bowl', 'Steak Burrito',
       'Steak Soft Tacos', 'Chicken Crispy Tacos', 'Chicken Soft Tacos',
       'Chicken Burrito', 'Canned Soda', 'Barbacoa Burrito',
       'Carnitas Burrito', 'Carnitas Bowl', 'Barbacoa Bowl',
       'Chicken Salad Bowl', 'Steak Bowl', 'Barbacoa Soft Tacos',
       'Veggie Burrito', 'Veggie Bowl', 'Steak Crispy Tacos',
       'Barbacoa Crispy Tacos', 'Veggie Salad Bowl',
       'Carnitas Soft Tacos', 'Chicken Salad', 'Canned Soft Drink',
       'Steak Salad Bowl', '6 Pack Soft Drink', 'Bowl', 'Burrito',
       'Crispy Tacos', 'Carnitas Crispy Tacos', 'Steak Salad',
       'Veggie Soft Tacos', 'Carnitas Salad Bowl', 'Barbacoa Salad Bowl',
       'Salad', 'Veggie Crispy Tacos', 'Veggie Salad', 'Carnitas Salad'],
      dtype=object)

In [23]:
chipotle_cleaned_missing = missing_cnt_pt_sorted(chipotle_cleaned)
chipotle_cleaned_missing

,df_col,sum_null,percent_null
0,choice_10,3372,99.881517
1,choice_9,3327,98.548578
2,choice_8,3197,94.697867
3,choice_7,2766,81.931280
4,choice_6,2042,60.485782
5,choice_5,1285,38.062796
6,choice_4,834,24.703791
7,choice_3,634,18.779621
8,choice_2,549,16.261848
9,order_id,0,0.000000


In [24]:
chipotle_cleaned.quantity.value_counts()

1    3191
2     162
3      17
4       6
Name: quantity, dtype: int64

In [25]:
chipotle_cleaned[(chipotle_cleaned['quantity']>1) & (chipotle_cleaned["Total_Price"] >20)].head()

,order_id,quantity,item_name,item_price,Total_Price,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,choice_10
2,2,2,Chicken Bowl,16.98,33.96,Tomatillo-Red Chili Salsa (Hot),Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None
94,60,2,Chicken Salad Bowl,22.50,45.00,Tomatillo Green Chili Salsa,Sour Cream,Cheese,Guacamole,None,None,None,None,None,None
103,67,2,Steak Burrito,17.98,35.96,Tomatillo-Red Chili Salsa (Hot),Rice,Cheese,Sour Cream,Lettuce,None,None,None,None,None
104,68,2,Chicken Burrito,17.50,35.00,Tomatillo Red Chili Salsa,Rice,Black Beans,Sour Cream,Cheese,Lettuce,None,None,None,None
106,70,2,Chicken Bowl,17.50,35.00,Fresh Tomato Salsa,Fajita Vegetables,Rice,Lettuce,None,None,None,None,None,None


In [26]:
chipotle_cleaned.select_dtypes('number').describe()

,order_id,quantity,item_price,Total_Price
count,3376.00000,3376.000000,3376.000000,3376.000000
mean,920.33205,1.063389,9.011321,9.953193
std,528.96664,0.283070,3.791337,8.071157
min,1.00000,1.000000,1.090000,1.090000
25%,466.75000,1.000000,8.690000,8.690000
50%,922.00000,1.000000,8.990000,8.990000
75%,1382.25000,1.000000,11.250000,11.250000
max,1834.00000,4.000000,35.250000,140.000000


In [27]:
chipotle_cleaned.select_dtypes('object').describe()

,item_name,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,choice_10
count,3376,3376,2827,2742,2542,2091,1334,610,179,49,4
unique,38,38,16,15,13,11,10,9,8,4,1
top,Chicken Bowl,Fresh Tomato Salsa,Rice,Rice,Cheese,Sour Cream,Lettuce,Lettuce,Lettuce,Lettuce,Lettuce
freq,726,1046,1340,829,1004,677,487,359,114,36,4


## EDA

In [28]:
cat_col=chipotle_cleaned.select_dtypes('object').columns
cat_col

Index(['item_name', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5',
       'choice_6', 'choice_7', 'choice_8', 'choice_9', 'choice_10'],
      dtype='object')

In [29]:
chipotle_cleaned[cat_col].head()

,item_name,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,choice_10
0,Izze,Clementine,None,None,None,None,None,None,None,None,None
1,Nantucket Nectar,Apple,None,None,None,None,None,None,None,None,None
2,Chicken Bowl,Tomatillo-Red Chili Salsa (Hot),Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None
3,Chicken Bowl,Fresh Tomato Salsa (Mild),Rice,Cheese,Sour Cream,Guacamole,Lettuce,None,None,None,None
4,Steak Burrito,Tomatillo Red Chili Salsa,Fajita Vegetables,Black Beans,Pinto Beans,Cheese,Sour Cream,Guacamole,Lettuce,None,None


In [30]:
num_col=chipotle_cleaned.select_dtypes('number').columns
num_col

Index(['order_id', 'quantity', 'item_price', 'Total_Price'], dtype='object')

In [31]:
def EDA_numeric(df):
    def eda(df,i):
        return {"Colname" : (df.columns)[i],
                "Mean " : df.iloc[:,i].mean(),
                "Med " : df.iloc[:,i].median(),
                "min " : df.iloc[:,i].min(),
                "max " : df.iloc[:,i].max(),
                "var " : df.iloc[:,i].var(),
                "sd " : df.iloc[:,i].std(),
                "skew " : df.iloc[:,i].skew(),
                "kurt " : df.iloc[:,i].kurt()}
    num_col = df.select_dtypes('number').columns
    eda_lst = [eda(df[num_col],i) for i in range(len(num_col))]
    eda_df = pd.DataFrame(eda_lst)
    return eda_df

In [32]:
chipotle_eda = EDA_numeric(chipotle_cleaned)

In [33]:
chipotle_eda

,Colname,Mean,Med,min,max,var,sd,skew,kurt
0,order_id,920.332050,922.00,1.00,1834.00,279805.706303,528.966640,0.017169,-1.194742
1,quantity,1.063389,1.00,1.00,4.00,0.080129,0.283070,5.329108,34.445579
2,item_price,9.011321,8.99,1.09,35.25,14.374234,3.791337,0.425318,5.301982
3,Total_Price,9.953193,8.99,1.09,140.00,65.143576,8.071157,6.491966,69.176131


# Dataset: Beer.txt

In [34]:
beer_df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/beer.txt',delimiter = " ")
beer_df.head()

,name,calories,sodium,alcohol,cost
0,Budweiser,144,15,4.7,0.43
1,Schlitz,151,19,4.9,0.43
2,Lowenbrau,157,15,0.9,0.48
3,Kronenbourg,170,7,5.2,0.73
4,Heineken,152,11,5.0,0.77


In [35]:
beer_missing = missing_cnt_pt_sorted(beer_df)
beer_missing #no missing data
#beer_df.isnull().sum()

,df_col,sum_null,percent_null
0,name,0,0.0
1,calories,0,0.0
2,sodium,0,0.0
3,alcohol,0,0.0
4,cost,0,0.0


In [36]:
beer_df.duplicated().sum() #no duplicates

0

In [37]:
beer_EDA_df = EDA_numeric(beer_df)

In [38]:
beer_EDA_df

,Colname,Mean,Med,min,max,var,sd,skew,kurt
0,calories,132.5500,144.000,68.00,175.00,917.102632,30.283702,-0.910703,-0.033435
1,sodium,14.9500,15.000,6.00,27.00,43.313158,6.581273,0.197447,-1.081145
2,alcohol,4.2400,4.600,0.90,5.50,1.183579,1.087924,-1.925627,4.015840
3,cost,0.4945,0.435,0.28,0.79,0.020984,0.144858,1.220011,0.399530


# Dataset: datasets.csv

In [39]:
dataset_df = pd.read_csv("https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/datasets.csv")

In [40]:
dataset_df.head()

,Package,Item,Title,Rows,Cols,n_binary,n_character,n_factor,n_logical,n_numeric,CSV,Doc
0,AER,Affairs,Fair's Extramarital Affairs Data,601,9,2,0,2,0,7,https://vincentarelbundock.github.io/Rdatasets...,https://vincentarelbundock.github.io/Rdatasets...
1,AER,ArgentinaCPI,Consumer Price Index in Argentina,80,2,0,0,0,0,2,https://vincentarelbundock.github.io/Rdatasets...,https://vincentarelbundock.github.io/Rdatasets...
2,AER,BankWages,Bank Wages,474,4,2,0,3,0,1,https://vincentarelbundock.github.io/Rdatasets...,https://vincentarelbundock.github.io/Rdatasets...
3,AER,BenderlyZwick,"Benderly and Zwick Data: Inflation, Growth and...",31,5,0,0,0,0,5,https://vincentarelbundock.github.io/Rdatasets...,https://vincentarelbundock.github.io/Rdatasets...
4,AER,BondYield,Bond Yield Data,60,2,0,0,0,0,2,https://vincentarelbundock.github.io/Rdatasets...,https://vincentarelbundock.github.io/Rdatasets...


In [41]:
dataset_df['CSV'] = dataset_df['CSV'].str.split('/').str[-1]

In [42]:
dataset_df['Doc'] = dataset_df['Doc'].str.split('/').str[-1]

In [43]:
dataset_df.head()

,Package,Item,Title,Rows,Cols,n_binary,n_character,n_factor,n_logical,n_numeric,CSV,Doc
0,AER,Affairs,Fair's Extramarital Affairs Data,601,9,2,0,2,0,7,Affairs.csv,Affairs.html
1,AER,ArgentinaCPI,Consumer Price Index in Argentina,80,2,0,0,0,0,2,ArgentinaCPI.csv,ArgentinaCPI.html
2,AER,BankWages,Bank Wages,474,4,2,0,3,0,1,BankWages.csv,BankWages.html
3,AER,BenderlyZwick,"Benderly and Zwick Data: Inflation, Growth and...",31,5,0,0,0,0,5,BenderlyZwick.csv,BenderlyZwick.html
4,AER,BondYield,Bond Yield Data,60,2,0,0,0,0,2,BondYield.csv,BondYield.html


In [44]:
dataset_missing = missing_cnt_pt_sorted(dataset_df)
dataset_missing

,df_col,sum_null,percent_null
0,Package,0,0.0
1,Item,0,0.0
2,Title,0,0.0
3,Rows,0,0.0
4,Cols,0,0.0
5,n_binary,0,0.0
6,n_character,0,0.0
7,n_factor,0,0.0
8,n_logical,0,0.0
9,n_numeric,0,0.0


In [45]:
dataset_df.duplicated().sum()

0

In [46]:
dataset_EDA_df = EDA_numeric(dataset_df)

In [47]:
dataset_EDA_df

,Colname,Mean,Med,min,max,var,sd,skew,kurt
0,Rows,3900.832555,105.5,2,1414593,1.597621e+09,39970.247883,27.576331,920.716828
1,Cols,13.086348,5.0,1,6831,2.957244e+04,171.966384,37.085842,1450.558862
2,n_binary,1.931155,0.0,0,624,3.033344e+02,17.416499,30.605284,1023.577648
3,n_character,0.302800,0.0,0,17,1.602946e+00,1.266075,7.275990,67.190156
4,n_factor,1.270128,0.0,0,64,1.020661e+01,3.194779,10.624950,168.407489
5,n_logical,0.030922,0.0,0,11,1.642505e-01,0.405278,19.147633,427.455648
6,n_numeric,11.429988,3.0,0,6830,2.955145e+04,171.905339,37.135071,1453.192625


# Dataset: US Crimes.csv

In [48]:
crime_df = pd.read_csv("https://gist.githubusercontent.com/GeekOnAcid/da022affd36310c96cd4/raw/9c2ac2b033979fcf14a8d9b2e3e390a4bcc6f0e3/us_nr_of_crimes_1960_2014.csv")

In [49]:
crime_df.head()

,Year,Total,Violent,Property,Murder,Rape,Robbery,Aggravated-Assault,Burglary,Larceny-Theft,Vehicle-Theft
0,1960,1887.2,160.9,1726.3,5.1,9.6,60.1,86.1,508.6,1034.7,183.0
1,1961,1906.1,158.1,1747.9,4.8,9.4,58.3,85.7,518.9,1045.4,183.6
2,1962,2019.8,162.3,1857.5,4.6,9.4,59.7,88.6,535.2,1124.8,197.4
3,1963,2180.3,168.2,2012.1,4.6,9.4,61.8,92.4,576.4,1219.1,216.6
4,1964,2388.1,190.6,2197.5,4.9,11.2,68.2,106.2,634.7,1315.5,247.4


In [50]:
missing_cnt_pt_sorted(crime_df)

,df_col,sum_null,percent_null
0,Year,0,0.0
1,Total,0,0.0
2,Violent,0,0.0
3,Property,0,0.0
4,Murder,0,0.0
5,Rape,0,0.0
6,Robbery,0,0.0
7,Aggravated-Assault,0,0.0
8,Burglary,0,0.0
9,Larceny-Theft,0,0.0


In [51]:
crime_df.loc[(crime_df['Total'] > 5000),"Total_cat"] = "VH"
crime_df.head()

,Year,Total,Violent,Property,Murder,Rape,Robbery,Aggravated-Assault,Burglary,Larceny-Theft,Vehicle-Theft,Total_cat
0,1960,1887.2,160.9,1726.3,5.1,9.6,60.1,86.1,508.6,1034.7,183.0,NaN
1,1961,1906.1,158.1,1747.9,4.8,9.4,58.3,85.7,518.9,1045.4,183.6,NaN
2,1962,2019.8,162.3,1857.5,4.6,9.4,59.7,88.6,535.2,1124.8,197.4,NaN
3,1963,2180.3,168.2,2012.1,4.6,9.4,61.8,92.4,576.4,1219.1,216.6,NaN
4,1964,2388.1,190.6,2197.5,4.9,11.2,68.2,106.2,634.7,1315.5,247.4,NaN


In [52]:
import numpy as np #better way to do the same as above

In [53]:
crime_df["Total_cat"] = np.where(crime_df["Total"]>=5000,"Very High",
                                 (np.where((crime_df["Total"]>=2500) & (crime_df["Total"]<5000),"High","Moderate")))

In [54]:
crime_df.head()

,Year,Total,Violent,Property,Murder,Rape,Robbery,Aggravated-Assault,Burglary,Larceny-Theft,Vehicle-Theft,Total_cat
0,1960,1887.2,160.9,1726.3,5.1,9.6,60.1,86.1,508.6,1034.7,183.0,Moderate
1,1961,1906.1,158.1,1747.9,4.8,9.4,58.3,85.7,518.9,1045.4,183.6,Moderate
2,1962,2019.8,162.3,1857.5,4.6,9.4,59.7,88.6,535.2,1124.8,197.4,Moderate
3,1963,2180.3,168.2,2012.1,4.6,9.4,61.8,92.4,576.4,1219.1,216.6,Moderate
4,1964,2388.1,190.6,2197.5,4.9,11.2,68.2,106.2,634.7,1315.5,247.4,Moderate


In [55]:
EDA_numeric(crime_df)

,Colname,Mean,Med,min,max,var,sd,skew,kurt
0,Year,1987.000000,1987.0,1960.0,2014.0,2.566667e+02,16.020820,0.000000,-1.200000
1,Total,4285.598182,4162.6,1887.2,5950.0,1.385519e+06,1177.080591,-0.399292,-0.885355
2,Violent,472.549091,475.8,158.1,758.1,2.595277e+04,161.098633,-0.289532,-0.388677
3,Property,3813.503636,3737.0,1726.3,5353.3,1.064542e+06,1031.766652,-0.360248,-0.951496
4,Murder,7.141818,7.3,4.5,10.2,3.583960e+00,1.893135,0.034253,-1.602866
5,Rape,28.690909,31.0,9.4,42.8,8.948121e+01,9.459451,-0.741017,-0.410698
6,Robbery,168.681818,172.1,58.3,272.7,3.612119e+03,60.100903,-0.216917,-0.885184
7,Aggravated-Assault,267.656364,283.8,85.7,441.8,1.000476e+04,100.023780,-0.157734,-0.664754
8,Burglary,1001.796364,945.0,508.6,1684.1,1.118518e+05,334.442460,0.299675,-1.191532
9,Larceny-Theft,2394.018182,2477.3,1034.7,3228.8,4.087666e+05,639.348563,-0.551616,-0.714863


In [56]:
val_ct_pt(crime_df,'Total_cat')

,counts,%
High,27,0.490909
Very High,22,0.400000
Moderate,6,0.109091


In [57]:
crime_df[crime_df["Total_cat"]=='High'].index #conditional selection

Int64Index([ 6,  7,  8,  9, 10, 11, 12, 13, 14, 37, 38, 39, 40, 41, 42, 43, 44,
            45, 46, 47, 48, 49, 50, 51, 52, 53, 54],
           dtype='int64')

In [58]:
crime_df[crime_df["Total_cat"]=='High']

,Year,Total,Violent,Property,Murder,Rape,Robbery,Aggravated-Assault,Burglary,Larceny-Theft,Vehicle-Theft,Total_cat
6,1966,2670.8,220.0,2450.9,5.6,13.2,80.8,120.3,721.0,1442.9,286.9,High
7,1967,2989.7,253.2,2736.5,6.2,14.0,102.8,130.2,826.6,1575.8,334.1,High
8,1968,3370.2,298.4,3071.8,6.9,15.9,131.8,143.8,932.3,1746.6,393.0,High
9,1969,3680.0,328.7,3351.3,7.3,18.5,148.4,154.5,984.1,1930.9,436.2,High
10,1970,3984.5,363.5,3621.0,7.9,18.7,172.1,164.8,1084.9,2079.3,456.8,High
11,1971,4164.7,396.0,3768.8,8.6,20.5,188.0,178.8,1163.5,2145.5,459.8,High
12,1972,3961.4,401.0,3560.4,9.0,22.5,180.7,188.8,1140.8,1993.6,426.1,High
13,1973,4154.4,417.4,3737.0,9.4,24.5,183.1,200.5,1222.5,2071.9,442.6,High
14,1974,4850.4,461.1,4389.3,9.8,26.2,209.3,215.8,1437.7,2489.5,462.2,High
37,1997,4927.3,611.0,4316.3,6.8,35.9,186.1,382.1,919.6,2891.8,505.7,High
